# Imports

In [1]:
from botiverse import chat_gui
from botiverse.bots import DeepTODS
from botiverse.models import BERTConfig, TRIPPYConfig
import gdown
import os

# Download necessary files to run the example

In [2]:
# Download ontology
f_id = '1tRprClzgNCtVks-SgZilBD7fdc5W3XxY'
file_url = f'https://drive.google.com/uc?export=download&confirm=pbef&id={f_id}'
output_file = 'ontology.json'
if not os.path.exists(output_file):
    gdown.download(file_url, output_file)
    print('Ontology downloaded successfully.')
else:
    print('Ontology already exists. Skipping download.')

Ontology already exists. Skipping download.


In [3]:
# Download label maps
f_id = '15guVrztM12KMMDp6jgu53ivUU945iRNp'
file_url = f'https://drive.google.com/uc?export=download&confirm=pbef&id={f_id}'
output_file = 'label_maps.json'
if not os.path.exists(output_file):
    gdown.download(file_url, output_file)
    print('Label maps downloaded successfully.')
else:
    print('Label maps already exists. Skipping download.')

Label maps already exists. Skipping download.


In [4]:
# Download DST weights trained on sim-R
f_id = '1POjBULmqxBrebvINfl989bskAstV3Zld'
file_url = f'https://drive.google.com/uc?export=download&confirm=pbef&id={f_id}'
output_file = 'model.pt'
if not os.path.exists(output_file):
    gdown.download(file_url, output_file)
    print('Model downloaded successfully.')
else:
    print('Model already exists. Skipping download.')

Model already exists. Skipping download.


# Const

In [5]:
CHATBOT_NAME = 'Tody'

DOMAINS = ["restaurant"]

ONTOLOGY_PATH = './ontology.json'

LABEL_MAPS_PATH = './label_maps.json'

POLICY = 'Priority' # Priority or Random

START = [
    {
        'utterance': 'Hi I am Tody, I can help you reserve a restaurant?',
        'slots': [],
        'system_act': {}
    }
]

TEMPLATES = [
    {
        'utterance': 'what type of food do you want and in what area?',
        'slots': ['restaurant-location', 'restaurant-category'],
        'system_act': {}
    },
    {
        'utterance': 'what is your preferred price range and rating?',
        'slots': ['restaurant-price_range', 'ressaurant-rating'],
        'system_act': {}
    },
    {
        'utterance': 'how many people will be in your party?',
        'slots': ['restaurant-num_people'],
        'system_act': {}
    },
    {
        'utterance': 'what time and date would you like to reserve a table for?',
        'slots': ['restaurant-time', 'restaurant-date'],
        'system_act': {}
    },
    {
        'utterance': 'May I suggest kfc restaurant?',
        'slots': ['restaurant-restaurant_name'],
        'system_act': {'restaurant-restaurant_name': ['kfc']}
    }

]

NON_REFERABLE_SLOTS = []

NON_REFERABLE_PAIRS = []

FROM_SCRATCH = True

MODEL_PATH = './model.pt'

TRIPPY_CONFIG = TRIPPYConfig(multiwoz=False)

BERT_CONFIG = BERTConfig()

# Build model & Load weights

In [6]:
tods = DeepTODS(CHATBOT_NAME, DOMAINS, ONTOLOGY_PATH, LABEL_MAPS_PATH, POLICY, START, TEMPLATES, NON_REFERABLE_SLOTS, NON_REFERABLE_PAIRS, FROM_SCRATCH, BERT_CONFIG, TRIPPY_CONFIG)
tods.load_dst_model(MODEL_PATH)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Model loaded successfully.


# Conversation

In [7]:
chat_gui = chat_gui(tods.infer)
chat_gui.run()

In [8]:
tods.reset()
print(tods.get_dialogue_state())

{}


In [9]:
response = tods.infer('')
print(tods.get_dialogue_state())
print(response)

{}
Hi I am Tody, I can help you reserve a restaurant?


In [10]:
response = tods.infer('Hi, I want to book a table for lunch.')
print(tods.get_dialogue_state())
print(response)

{'restaurant-meal': 'lunch'}
what type of food do you want and in what area?


In [11]:
response = tods.infer('I would like to have egyptian food in cairo.')
print(tods.get_dialogue_state())
print(response)

{'restaurant-meal': 'lunch', 'restaurant-category': 'egyptian', 'restaurant-location': 'cairo'}
what is your preferred price range and rating?


In [12]:
response = tods.infer('I want a cheap restaurant with good rating.')
print(tods.get_dialogue_state())
print(response)

{'restaurant-meal': 'lunch', 'restaurant-category': 'egyptian', 'restaurant-location': 'cairo', 'restaurant-price_range': 'cheap', 'restaurant-rating': 'good'}
how many people will be in your party?


In [13]:
response = tods.infer('I have 4 people in my party.')
print(tods.get_dialogue_state())
print(response)

{'restaurant-meal': 'lunch', 'restaurant-category': 'egyptian', 'restaurant-location': 'cairo', 'restaurant-price_range': 'cheap', 'restaurant-rating': 'good', 'restaurant-num_people': '4'}
what time and date would you like to reserve a table for?


In [15]:
response = tods.infer('4 am on the tuesday.')
print(tods.get_dialogue_state())
print(response)

{'restaurant-meal': 'lunch', 'restaurant-category': 'egyptian', 'restaurant-location': 'cairo', 'restaurant-price_range': 'cheap', 'restaurant-rating': 'good', 'restaurant-num_people': '4', 'restaurant-date': 'tuesday', 'restaurant-time': '4 am'}
May I suggest kfc restaurant?


In [16]:
response = tods.infer('No, I hate kfc.')
print(tods.get_dialogue_state())
print(response)

{'restaurant-meal': 'lunch', 'restaurant-category': 'egyptian', 'restaurant-location': 'cairo', 'restaurant-price_range': 'cheap', 'restaurant-rating': 'good', 'restaurant-num_people': '4', 'restaurant-date': 'tuesday', 'restaurant-time': '4 am'}
May I suggest kfc restaurant?


In [17]:
response = tods.infer('Yes, I would love that.')
print(tods.get_dialogue_state())
print(response)

{'restaurant-meal': 'lunch', 'restaurant-category': 'egyptian', 'restaurant-location': 'cairo', 'restaurant-price_range': 'cheap', 'restaurant-rating': 'good', 'restaurant-num_people': '4', 'restaurant-date': 'tuesday', 'restaurant-time': '4 am', 'restaurant-restaurant_name': '§§kfc'}
None
